In [0]:
%pip install azure-storage-blob

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b885cdb1-4633-4bed-a823-79b8bd5dd970
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
connection_string = "DefaultEndpointsProtocol=https;AccountName=datalakedemo001;AccountKey=ndrhiH6jpzITegf7mGnuFB14JGKcDJqSedz/QeIoFxRRpCbql9r5TtDn94WrsC2i5M9Vqv5Puylm+ASt/s0Akw==;EndpointSuffix=core.windows.net"
container_name = "classwork"
blob_name = "HR_Employee.csv"


from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AzureBlobSparkReadStream") \
    .getOrCreate()

# Set the Azure Storage account key
spark.conf.set(
    "fs.azure.account.key.datalakedemo001.blob.core.windows.net",
    "ndrhiH6jpzITegf7mGnuFB14JGKcDJqSedz/QeIoFxRRpCbql9r5TtDn94WrsC2i5M9Vqv5Puylm+ASt/s0Akw=="
)
blob_url = f"wasbs://{container_name}@{connection_string.split(';')[1].split('=')[1]}.blob.core.windows.net/{blob_name}"
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(blob_url)



# Read the CSV file as a streaming DataFrame
df1 = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema(df.schema) \
    .load(blob_url)

# # Display the streaming DataFrame
df1.printSchema()




root
 |-- EmployeeID: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- JobInvolvement: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: string (nullable = true)
 |-- Hourlyrate: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Salaryhike: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Workex: integer (nullable = true)
 |-- YearsSinceLastPromotion: integer (nullable = true)
 |-- EmpSatisfaction: string (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: string (nullable = true)
 |-- Performance_Rating: string (nul

In [0]:
from pyspark.sql.functions import *

# Perform basic transformations
df2 = df1.filter(col("age") >= 30)
df2 = df2.withColumnRenamed("department", "dept")


In [0]:
df3 = df2.groupBy("dept").agg(
    sum("Income").alias("total_salary"),
    avg("Age").alias("average_age"),
    count("*").alias("employee_count")
)


In [0]:

# Define the Cosmos DB connection details
cosmos_db_endpoint = "https://classworks.documents.azure.com:443/"
cosmos_db_master_key = "xHoCCouttJqTLi0OWjpfptC0RqyccltDPi6lFbhtaq1jUbcrXD3bnnXMoIG2rDtlfDnb0CvfoEK9ACDbnmuoNQ=="
cosmos_db_database = "BlobToDb"
cosmos_db_collection = "ContainerCW"

# Define the Cosmos DB sink
cosmos_db_sink = spark._jvm.com.microsoft.azure.cosmosdb.spark.config.Config(
    endpoint=cosmos_db_endpoint,
    masterKey=cosmos_db_master_key,
    database=cosmos_db_database,
    collection=cosmos_db_collection
)

# Write the aggregated DataFrame to Cosmos DB
df3.writeStream \
    .format("cosmosdb") \
    .options(**cosmos_db_sink) \
    .option("checkpointLocation", "/tmp/checkpoint") \
    .start()
